In [33]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry
import psycopg2
from sqlalchemy import create_engine
import geoalchemy2
import requests
import zipfile

In [ ]:
#database connection parameters
db_params = {
    "host": "34.30.71.239",
    "port": "5432",
    "dbname": "gis_data",
    "user": "",
    "password": ""
}

#db connection string
db_string = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
engine = create_engine(db_string)

In [24]:
#define cooling center table variable and load

cooling_options = "cooling_options"
cooling_options_df = pd.read_sql(f"SELECT * FROM {cooling_options}", engine)

In [25]:
#visualize table
print(cooling_options_df)

      id                                           geometry  \
0      1  010100002023690000D03B4ED151651B4152499D9CCA06...   
1      2  010100002023690000F0E461E1F6661B412041F18B9A06...   
2      3  010100002023690000D0E53F64C4221B41A067B31610FC...   
3      4  0101000020236900003080B780EE371B410EE02DC854FC...   
4      5  010100002023690000D076BEDF375F1B41E09C11C506FD...   
..   ...                                                ...   
313  314  010100002023690000E03FA41F58D21C415A643BBF1302...   
314  315  01010000202369000080C954011DCA1C419EEFA7861303...   
315  316  010100002023690000E0240681FBB91C41EE7C3F9DC2FA...   
316  317  010100002023690000D07F48BFB2B31C416ABC74BB1A0C...   
317  318  010100002023690000D012F24126151D417C3F35FE91FC...   

                                                name                address  \
0                                    Camp Ihduhapi 1    3425 Ihduhapi Trail   
1                                    Camp Ihduhapi 2    3425 Ihduhapi Trail   
2     

In [26]:
#view specifically 'fee' and 'inhennepin' fields
print(cooling_options_df[['fee','inhennepin']])

     fee inhennepin
0    Yes   Hennepin
1    Yes   Hennepin
2    Yes   Hennepin
3    Yes   Hennepin
4     No   Hennepin
..   ...        ...
313   No   Hennepin
314   No   Hennepin
315   No   Hennepin
316  Yes   Hennepin
317   No   Hennepin

[318 rows x 2 columns]


In [27]:
#remove any cooling center where fee attribute = yes and inhennepin = no
cooling_options_df_filtered = cooling_options_df[(cooling_options_df["inhennepin"] != "No") & (cooling_options_df["fee"] != "Yes")]

In [28]:
#visualize filtered table
print(cooling_options_df_filtered)

      id                                           geometry  \
4      5  010100002023690000D076BEDF375F1B41E09C11C506FD...   
5      6  010100002023690000806132D560891B41A2B4377C41FA...   
6      7  010100002023690000E02D90A0E05A1B41984C150CD0F9...   
7      8  010100002023690000C07493D84EE11B41A679C7CD21FA...   
8      9  010100002023690000E08C28ADE79A1B41CEAACFCD80FA...   
..   ...                                                ...   
312  313  010100002023690000B0A44EC0BF4C1D4188D2DE8C90FC...   
313  314  010100002023690000E03FA41F58D21C415A643BBF1302...   
314  315  01010000202369000080C954011DCA1C419EEFA7861303...   
315  316  010100002023690000E0240681FBB91C41EE7C3F9DC2FA...   
317  318  010100002023690000D012F24126151D417C3F35FE91FC...   

                                                name                address  \
4                                       Canary Beach        1550 Canary Ln.   
5                                  Casco Point Beach  2879 Casco Point Road   
6     

In [29]:
#double check filtered columns
print(cooling_options_df_filtered[['fee','inhennepin']])

    fee inhennepin
4    No   Hennepin
5    No   Hennepin
6    No   Hennepin
7    No   Hennepin
8    No   Hennepin
..   ..        ...
312  No   Hennepin
313  No   Hennepin
314  No   Hennepin
315  No   Hennepin
317  No   Hennepin

[250 rows x 2 columns]


In [47]:
#make the cooling options dataframe a geodataframe with geopandas
cooling_options_gdf = gpd.GeoDataFrame(cooling_options_df_filtered, geometry=gpd.points_from_xy(cooling_options_df_filtered["longitude"], cooling_options_df_filtered["latitude"]))

In [34]:
#double check by verifying all cooling center points are with hennepin county polygon

#download mn counties shapefile
url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_counties_in_minnesota/shp_bdry_counties_in_minnesota.zip"
response = requests.get(url)
with open("mn_county.zip", "wb") as f:
    f.write(response.content)

# Extract the GeoPackage file
with zipfile.ZipFile("mn_county.zip", "r") as zip_ref:
    zip_ref.extractall("mn_county")

In [ ]:
#load the downloaded shapefile with geopandas and print to view
mn_county_path = "C:\Users\KOlso\OneDrive\Documents\ArcGIS\Projects\GIS5571_Lab2\mn_county\mn_county_boundaries_500.shp"
mn_county = gpd.read_file(mn_county_path)

#only need county name
print(mn_county['CTY_NAME'])

0      Lake of the Woods
1                Kittson
2                 Roseau
3            Koochiching
4               Marshall
             ...        
112                 Cook
113                 Cook
114                 Cook
115                 Lake
116                 Lake
Name: CTY_NAME, Length: 117, dtype: object


In [43]:
#filter to just be hennepin county
hennepin_county = mn_county[(mn_county["CTY_NAME"] == "Hennepin")]
print(hennepin_county)
hennepin_county.explore()

            AREA      PERIMETER  CTYONLY_  CTYONLY_ID  COUN  CTY_NAME  \
49  1.570587e+09  190077.767485      51.0        50.0    27  Hennepin   

   CTY_ABBR  CTY_FIPS  MaxSimpTol  MinSimpTol     Shape_Leng    Shape_Area  \
49     HENN        53       500.0       500.0  176369.888097  1.569322e+09   

                                             geometry  
49  POLYGON ((459105.693 5010507.644, 459790.33 50...  


Matplotlib is building the font cache; this may take a moment.


In [50]:
#to check if all cooling centers are within hennepin county, first check both df using the same CRS

#set cooling_centers_gdf CRS
cooling_options_gdf = cooling_options_gdf.set_crs(epsg=32615) #UTM 15N

hennepin_county = hennepin_county.to_crs(epsg=32615)  #UTM 15N
cooling_options = cooling_options_gdf.to_crs(hennepin_county.crs)

In [53]:
#run check to see if points are inside hennepin
cooling_options["inside_hennepin"] = cooling_options.geometry.within(hennepin_county.geometry.iloc[0])

inside_count = cooling_options["inside_hennepin"].sum()
total_count = len(cooling_options)

print(f"{inside_count} out of {total_count} points are within Hennepin County.")


0 out of 250 points are within Hennepin County.


In [ ]:
#replace existing cooling options table in database with the now filtered one

db_connection = "postgresql://**:**@34.30.71.239:5432/gis_data"
engine = create_engine(db_connection)

cooling_options_gdf.to_postgis(name = "cooling_options", con=engine, if_exists="replace", index = False)
